# Callbacks · Design Patterns & Best Practices (ADK)

Callbacks hook into the agent lifecycle to **observe**, **modify**, or **override** behavior.

This notebook explains **eight design patterns** with **visual diagrams** and practical notes, plus a concise **best practices** section.

> Note: Mermaid diagrams render in many Markdown viewers (VS Code, GitHub, Notion). In plain Jupyter, a Mermaid extension may be required.

## Callback map (where each one fires)

```mermaid
flowchart LR
  U[User] --> R[Runner]
  R --> A[Agent]
  A -->|before_agent| A
  A -->|before_model| L[LLM]
  L -->|after_model| A
  A -->|before_tool| T[Tool]
  T -->|after_tool| A
  A -->|after_agent| R --> U
```

- **before_agent / after_agent** wrap the agent’s turn.
- **before_model / after_model** wrap the LLM call.
- **before_tool / after_tool** wrap each tool execution.

**Override rule (doc-standard):**
- Returning a value from a `before_*` callback **skips** the step.
- Returning a value from an `after_*` callback **replaces** the result.

| Callback | Skip/Replace Value |
|---|---|
| before_agent | `types.Content` |
| before_model | `LlmResponse` |
| before_tool | `dict` |
| after_agent | `types.Content` |
| after_model | `LlmResponse` |
| after_tool | `dict` |

## 1) Guardrails & Policy Enforcement

**Goal:** Intercept requests **before** they reach the LLM or a tool and enforce rules (topics, profanity, PII, quotas).

```mermaid
flowchart LR
   A[before_model] -->|check policy| C{Violation?}
   C -->|yes| X[Return standard LlmResponse<br/>block request]
   C -->|no| M[Call LLM]
```

**Use when:** You must block or standardize responses under specific policies.

**Notes:**
- Inspect `LlmRequest` (prompt contents, system instruction).
- For tools, inspect args in `before_tool`.
- Log flags to `state` (e.g., `state['temp:blocked'] = True`).

## 2) Dynamic State Management

**Goal:** Read/write `state` in callbacks to make behavior context-aware and pass data between lifecycle steps.

```mermaid
flowchart TB
  subgraph Session State
    S1[plain keys]:::session
    S2[user:*]:::user
    S3[app:*]:::app
    S4[temp:*]:::temp
  end
  A[Callbacks] -->|read/write| S1
  A --> S2
  A --> S3
  A --> S4

classDef session fill:#eef
classDef user fill:#efe
classDef app fill:#fee
classDef temp stroke-dasharray:4 4
```

**Use when:** You need to stash small facts (ids, preferences), track flags, or feed later steps.

**Notes:**
- Writes are captured as `state_delta` and persisted by the runtime.
- Prefer specific keys; avoid large opaque blobs.
- Choose the smallest scope (temp → session → user → app).

## 3) Logging & Monitoring

**Goal:** Add observability. Log inputs/outputs and metadata at exact lifecycle points.

```mermaid
sequenceDiagram
  participant A as Agent
  participant BM as before_model
  participant M as LLM
  participant AM as after_model
  A->>BM: request
  BM-->>BM: log (agent, turn id, prompt)
  BM->>M: call
  M-->>AM: response
  AM-->>AM: log (latency, tokens, truncation)
```

**Use when:** You need structured traces for debugging or analytics.

**Notes:** Keep logging light; avoid blocking I/O inside callbacks.

## 4) Caching

**Goal:** Skip redundant LLM calls or tool executions.

```mermaid
flowchart LR
  A[before_model] -->|compute key| H{Cache hit?}
  H -- yes --> X[Return cached LlmResponse]
  H -- no --> M[Call LLM]
  M --> AM[after_model]
  AM -->|store result| C[(Cache)]
```

**Use when:** Results are deterministic/cheap to reuse, or rate-limits are tight.

**Notes:**
- Stable keys (normalized prompt/args), TTLs, and a bypass flag (`temp:cache_bypass`).
- Implement similarly for tools with `before_tool`/`after_tool`.

## 5) Request/Response Modification

**Goal:** Adjust data just before send or after receive.

```mermaid
flowchart LR
  P[before_model] -->|tweak system instruction<br/>or prompt| M[Call LLM]
  M --> Q[after_model]
  Q -->|normalize/format| O[Final output]
```

**Use when:** You need to insert guidance (e.g., language preference) or reshape the model output.

**Notes:**
- Modify `LlmRequest` in `before_model`.
- Reformat/clean `LlmResponse` in `after_model`.
- For tools, adjust args in `before_tool`, normalize dicts in `after_tool`.

## 6) Conditional Skipping of Steps

**Goal:** Prevent standard operations (agent run, LLM call, or tool call) based on conditions.

```mermaid
flowchart LR
   B[before_tool] -->|quota check| C{Exceeded?}
   C -->|yes| X[Return dict<br/>skip tool]
   C -->|no| T[Run tool]
   
```

**Use when:** A precondition fails (auth, quota, validation) and you want a clean, immediate answer.

**Notes:** The returned value becomes the step’s result; downstream callbacks see this value.

## 7) Tool-Specific Actions (Authentication & Summarization Control)

**Goal:** Handle tool-only behaviors: credential requests and how results are summarized.

```mermaid
sequenceDiagram
  participant BT as before_tool
  participant S as Secure API
  participant AT as after_tool
  BT->>BT: check token in state
  alt missing token
    BT-->>BT: request_credential(...)
  end
  BT->>S: call with auth
  S-->>AT: result (JSON)
  AT-->>AT: set skip_summarization = True (optional)
```

**Use when:** A tool needs a token flow or you want to pass raw JSON back (not LLM-summarized).

**Notes:**
- Use `tool_context.request_credential(...)` if credentials are missing.
- Set `tool_context.actions.skip_summarization = True` to bypass LLM summarization for that tool result.

## 8 Artifact Handling

**Goal:** Save or load files/blobs during the agent lifecycle (reports, logs, intermediate data).

```mermaid
flowchart LR
   AT[after_tool] -->|save_artifact file| AR[(Artifacts)]
   BA[before_agent] -->|load_artifact file| AR
   
   style AT fill:#e1f5fe
   style BA fill:#f3e5f5
   style AR fill:#e8f5e8
```

**Use when:** You want durable outputs available across turns/sessions.

**Notes:**
- Use `save_artifact` / `load_artifact` from the provided contexts.
- Artifact changes are tracked via `artifact_delta` in events.

---
## Best Practices (Concise)

**Design principles**
- Keep callbacks focused: one clear purpose per callback.
- Prefer small, composable callbacks over monoliths.

**Performance**
- Callbacks run in the agent loop; keep them fast.
- Avoid heavy network/CPU work; offload if needed.

**Error handling**
- Use try/except; log errors with enough context.
- Decide whether to proceed, return a fallback, or stop.

**State management**
- Be deliberate with `state` keys; pick the smallest scope (temp/session/user/app).
- Writes are persisted via `state_delta`; avoid direct mutation outside contexts.

**Reliability**
- Aim for idempotency when side effects are possible.
- Make cache and policy checks deterministic.

**Testing & documentation**
- Unit test with mock contexts; run end-to-end tests in Runner/Web UI.
- Name callbacks clearly; include docstrings explaining when they run and what they change.

**Use the right context**
- Agent/model callbacks use `CallbackContext`.
- Tool callbacks use `ToolContext` (for auth, skip_summarization, artifacts).